<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a custom object detection model with TensorFlow Lite Model Maker

In this colab notebook, you'll learn how to use the [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/guide/model_maker) to train a custom object detection model to detect Android figurines and how to put the model on a Raspberry Pi.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset. Retraining a TensorFlow Lite model with your own custom dataset reduces the amount of training data required and will shorten the training time.


## Preparation

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library you'll use for evaluation.

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

Import the required packages.

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### Prepare the dataset

We start with downloading the dataset.

In [ ]:
# !wget https://github.com/Talonee/Portal/raw/main/pi/grass.zip
# !unzip -q grass.zip


Archive:  /content/drive/MyDrive/grass.zip
replace sdp/validate/104.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'grass/train',
    'grass/train',
    ['grass']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'grass/validate',
    'grass/validate',
    ['grass']
)

In [ ]:
# importing the module
from PIL import Image
import os
from os import listdir
from os.path import isfile, join
import re

dir = "grass/train/"

for f in listdir(dir):
    if (isfile(f"{dir}{f}") and re.search("jpg$", f)):
        # importing the image 
        im = Image.open(f"{dir}{f}")

        # re-convert to jpg, pure jpg downloads from google isn't recognized
        rgb_im = im.convert("RGB")

        # exporting the image
        rgb_im.save(f"{dir}{f}")

In [ ]:
print(f"{len(train_data)}\n{len(val_data)}")

111
23


### Step 2: Select a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In this notebook, we use EfficientDet-Lite0 to train our model. You can choose other model architectures depending on whether speed or accuracy is more important to you.

In [ ]:
spec = model_spec.get('efficientdet_lite1')

### Step 3: Train the TensorFlow model with the training data.

* Set `epochs = 20`, which means it will go through the training dataset 20 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 4` here so you will see that it takes 15 steps to go through the 62 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
27/27 [==============================] - 49s 581ms/step - det_loss: 1.6440 - cls_loss: 1.0867 - box_loss: 0.0111 - reg_l2_loss: 0.0706 - loss: 1.7146 - learning_rate: 0.0065 - gradient_norm: 2.8506 - val_det_loss: 1.3402 - val_cls_loss: 0.9522 - val_box_loss: 0.0078 - val_reg_l2_loss: 0.0706 - val_loss: 1.4108
Epoch 2/20
27/27 [==============================] - 12s 468ms/step - det_loss: 1.2665 - cls_loss: 0.8259 - box_loss: 0.0088 - reg_l2_loss: 0.0706 - loss: 1.3371 - learning_rate: 0.0049 - gradient_norm: 4.1240 - val_det_loss: 0.6460 - val_cls_loss: 0.3756 - val_box_loss: 0.0054 - val_reg_l2_loss: 0.0706 - val_loss: 0.7167
Epoch 3/20
27/27 [==============================] - 12s 459ms/step - det_loss: 0.8556 - cls_loss: 0.5203 - box_loss: 0.0067 - reg_l2_loss: 0.0706 - loss: 0.9262 - learning_rate: 0.0048 - gradient_norm: 4.9807 - val_det_loss: 0.4719 - val_cls_loss: 0.2594 - val_box_loss: 0.0042 - val_reg_l2_loss: 0.0707 - val_loss: 0.5426
Epoch 4/20
27/27 [=============

### Step 4. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 10 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 1 step to go through the 10 images in the validation dataset.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [ ]:
model.evaluate(val_data)

1/1 [==============================] - 8s 8s/step



{'AP': 0.6638197,
 'AP50': 0.85678923,
 'AP75': 0.7326733,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.66383123,
 'ARmax1': 0.6826087,
 'ARmax10': 0.7,
 'ARmax100': 0.70434785,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.70434785,
 'AP_/grass': 0.6638197}

In [ ]:
tflite_name='grass1.tflite'

### Step 5: Export as a TensorFlow Lite model.

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is [full integer quantization](https://www.tensorflow.org/lite/performance/post_training_integer_quant). This allows the TensorFlow Lite model to be smaller, run faster on Raspberry Pi CPU and also compatible with the Google Coral EdgeTPU.

In [ ]:
model.export(export_dir='.', tflite_filename=tflite_name)

### Step 6:  Evaluate the TensorFlow Lite model.

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
model.evaluate_tflite(tflite_name, val_data)

23/23 [==============================] - 121s 5s/step



{'AP': 0.6731102,
 'AP50': 0.8890146,
 'AP75': 0.7326733,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.6731102,
 'ARmax1': 0.6869565,
 'ARmax10': 0.70434785,
 'ARmax100': 0.70434785,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.70434785,
 'AP_/grass': 0.6731102}

In [ ]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download(tflite_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>